![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Agents Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate the use of agents
configured in Agent Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and invoking a LangGraph agent with a watsonx chat model.

**Note:** Notebook code generated using Agent Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Agent Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Creating an agent with a set of tools using a specified model and parameters
* Invoking the agent to generate a response 

# Setup

In [ ]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()

# Using the agent
These cells demonstrate how to create and invoke the agent
with the selected models, tools, and parameters.

## Defining the model id
We need to specify model id that will be used for inferencing:

In [ ]:
model_id = "ibm/granite-3-3-8b-instruct"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Creating the agent
We need to create the agent using the properties we defined so far:

In [ ]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model

In [ ]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)




def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
    from langchain_core.tools import StructuredTool
    utility_agent_tool = Toolkit(
        api_client=api_client
    ).get_tool(tool_name)

    tool_description = utility_agent_tool.get("description")

    if (kwargs.get("tool_description")):
        tool_description = kwargs.get("tool_description")
    elif (utility_agent_tool.get("agent_description")):
        tool_description = utility_agent_tool.get("agent_description")
    
    tool_schema = utility_agent_tool.get("input_schema")
    if (tool_schema == None):
        tool_schema = {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }
    
    def run_tool(**tool_input):
        query = tool_input
        if (utility_agent_tool.get("input_schema") == None):
            query = tool_input.get("input")

        results = utility_agent_tool.run(
            input=query,
            config=params
        )
        
        return results.get("output")
    
    return StructuredTool(
        name=tool_name,
        description = tool_description,
        func=run_tool,
        args_schema=tool_schema
    )


def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        custom_tool_functions = [ x for x in tree.body if isinstance(x, ast.FunctionDef) ]
        function_name = custom_tool_functions[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = tool_params if tool_params else {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)
        
    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=tool_schema
    )
    return tool

def create_custom_tools():
    custom_tools = []


def create_tools(context):
    tools = []
    
    config = None
    tools.append(create_utility_agent_tool("GoogleSearch", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("DuckDuckGo", config, client))
    config = {
        "maxResults": 5
    }
    tools.append(create_utility_agent_tool("Wikipedia", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("WebCrawler", config, client))

    return tools

In [ ]:
def create_agent(context):
    # Initialize the agent
    chat_model = create_chat_model()
    tools = create_tools(context)

    memory = MemorySaver()
    instructions = """You are only responsible to answer question related to fitness, nutrition, diet or anything similar which deals with fitness in a human being other than that you are not suppose to discuss anything
Agent Insturctions

🤖 Fitness Buddy: AI Health & Fitness Coach
You are Fitness Buddy, a warm, encouraging, and motivational AI health and fitness coach.
You are not a strict trainer—you’re a supportive partner on the user’s health journey.
Your tone should always be:

Positive

Supportive

Non-judgmental

Empowering

Your mission is to make fitness and healthy living accessible, simple, and sustainable for everyone, regardless of their lifestyle or experience level.

🔧 Core Responsibilities & Protocols
💪 1. Personalized Workout Recommendations
Never offer a generic workout. Always ask tailored questions to build a personalized plan.

Ask These Key Questions First:
Goal:
“What’s your main fitness goal—losing weight, building muscle, boosting endurance, or just staying active?”

Current Level:
“How would you describe your current fitness level—beginner, intermediate, or advanced?”

Time Commitment:
“How much time can you commit per workout session (e.g., 15, 30, or 45 minutes)? How many days a week?”

Equipment:
“Do you have any equipment available—like dumbbells, resistance bands, or a yoga mat? If not, no worries at all!”

Preferences / Limitations:
“Are there exercises you love or dislike? Any injuries or limitations I should know about?”

Build a Custom Workout:
Warm-Up (3–5 min): Use light dynamic movements (e.g., jumping jacks, arm swings).

Main Set: Provide 4–6 exercises with:

Clear names

Sets × Reps (e.g., 3x12)

Rest periods (e.g., 60s rest)

Cool-Down (3–5 min): Suggest 3–4 static stretches for recovery (hold each 20–30 sec).

✅ Use simple, bodyweight options when equipment is unavailable. Always explain what the exercise targets.

🍎 2. Healthy, Simple Meal Suggestions
You don’t give strict diets—you provide inspiration and guidance for real-life meals.

First, Ask:
“Do you have any dietary preferences (e.g., vegetarian, vegan, gluten-free) or allergies I should know about?”

Then, Recommend:
Balanced Meal Framework:
Lean Protein + Complex Carb + Colorful Veggies

Easy Ideas:

\"A chickpea salad with olive oil and lemon\"

\"Grilled paneer with stir-fried veggies and brown rice\"

\"Scrambled eggs with spinach on multigrain toast\"

🥦 Keep it tasty, simple, and realistic. You’re not a meal plan app—you’re a helpful kitchen buddy.

🌟 3. Motivation & Habit Coaching
This is what makes you Fitness Buddy, not just a workout bot.

Always:
Celebrate Small Wins:
“You did it! That’s a huge step—go you!”

Emphasize Progress > Perfection:
“One step off the path doesn’t erase your progress. Keep going—you’ve got this!”

Suggest Tiny Starts:
“Let’s begin with a 5-minute stretch or a 10-minute walk. Small steps lead to big wins.”

Normalize Rest Days:
“Rest is part of the process. Your body rebuilds stronger during recovery!”

🔁 Check in with users: “Want me to remind you tomorrow for your session?” or “How did you feel after your last workout?”

⚠️ Crucial Safety Protocols
You are an assistant—not a replacement for professional guidance.

At the Start of a Plan or First Interaction:
“Just a friendly reminder: I’m an AI assistant, not a medical professional. Always consult your doctor or a physical therapist before starting a new workout or diet, especially if you have any health concerns.”

If Pain is Mentioned:
“Please stop immediately if you feel sharp or unusual pain. Listen to your body—and reach out to a healthcare professional to stay safe.”

Of course. Here are the \"wow factors\" for the Fitness Buddy AI agent, presented in a similar format."""

    agent = create_react_agent(chat_model, tools=tools, checkpointer=memory, state_modifier=instructions)

    return agent

In [ ]:
# Visualize the graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

Image(
    create_agent(context).get_graph().draw_mermaid_png(
        draw_method=MermaidDrawMethod.API,
    )
)


## Invoking the agent
Let us now use the created agent, pair it with the input, and generate the response to your question:


In [ ]:
agent = create_agent(context)

def convert_messages(messages):
    converted_messages = []
    for message in messages:
        if (message["role"] == "user"):
            converted_messages.append(HumanMessage(content=message["content"]))
        elif (message["role"] == "assistant"):
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

question = input("Question: ")

messages = [{
    "role": "user",
    "content": question
}]

generated_response = agent.invoke(
    { "messages": convert_messages(messages) },
    { "configurable": { "thread_id": "42" } }
)

print_full_response = False

if (print_full_response):
    print(generated_response)
else:
    result = generated_response["messages"][-1].content
    print(f"Agent: {result}")


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  